In [4]:

import pandas as pd
import numpy as np
from numpy import random
from sklearn.model_selection import train_test_split

In [8]:
import pandas as pd

def load_data_with_pandas(file_path):
    data = pd.read_csv(file_path)
    return data

data_bumper = load_data_with_pandas('./data/data/output/bumper/bumper_data.csv',)
data_door_ed = load_data_with_pandas('./data/data/output/door/door_ed_data.csv')
data_door_scratch = load_data_with_pandas('./data/data/output/door/door_scratch_data.csv')
data_fender = load_data_with_pandas('./data/data/output/fender/fender_ed_data.csv',)
data_frame_ed = load_data_with_pandas('./data/data/output/frame/frame_ed_data.csv')
data_frame_hd = load_data_with_pandas('./data/data/output/frame/frame_hd_data.csv')
data_frame_sealf = load_data_with_pandas('./data/data/output/frame/frame_sealf_data.csv')
data_frame_seamf = load_data_with_pandas('./data/data/output/frame/frame_seamf_data.csv')


# 데이터를 하나의 데이터프레임으로 결합
data = pd.concat([
    data_bumper,
    data_door_ed,
    data_door_scratch,
    data_fender,
    data_frame_ed,
    data_frame_hd,
    data_frame_sealf,
    data_frame_seamf
], axis=0)  # axis=0: 행 방향으로 결합

# 결과 출력
print(len(data))  # 총 데이터 개수
print(data)


7381
                                                   path  quality part  \
0     .\data\data\source_data\bumper\scratch\205_101...        1   범퍼   
1     .\data\data\source_data\bumper\scratch\205_101...        1   범퍼   
2     .\data\data\source_data\bumper\scratch\205_101...        1   범퍼   
3     .\data\data\source_data\bumper\scratch\205_101...        1   범퍼   
4     .\data\data\source_data\bumper\scratch\205_101...        1   범퍼   
...                                                 ...      ...  ...   
1044  .\data\data\source_data\frame\Seam_failure\207...        0  프레임   
1045  .\data\data\source_data\frame\Seam_failure\207...        0  프레임   
1046  .\data\data\source_data\frame\Seam_failure\207...        0  프레임   
1047  .\data\data\source_data\frame\Seam_failure\207...        0  프레임   
1048  .\data\data\source_data\frame\Seam_failure\207...        0  프레임   

                                                 bboxes  
0                                                    []  
1 

In [ ]:
import random

# 데이터 개수
total_data = len(data)  # 예를 들어 7400개
batch_size = 100  # 각 배치 크기

# 배치 수 계산
num_batches = total_data // batch_size

# 중복되지 않는 랜덤 샘플을 추출하여 배치로 나누기
random_indices = random.sample(range(total_data), total_data)  # 전체 데이터를 중복 없이 랜덤하게 섞기

# 데이터 나누기
batches = [random_indices[i * batch_size: (i + 1) * batch_size] for i in range(num_batches)]

# 마지막 배치가 200개보다 적을 경우 처리
if total_data % batch_size != 0:
    batches.append(random_indices[num_batches * batch_size:])

# 결과 확인
# for i, batch in enumerate(batches):
    # print(f"Batch {i + 1}: {len(batch)} items")

print(batches[2])


In [ ]:
from PIL import Image
import numpy as np
import ast  # 문자열을 리스트로 변환하기 위해 사용

# 이미지 데이터 로드 및 배열 생성
x = []  # 이미지 데이터를 저장할 리스트
y = []  # 라벨을 저장할 리스트

# 각 배치에 대해 이미지 데이터 로드
count = 0  # 배치 완료 횟수를 추적
for batch in batches:
    y_batch = []
    x_batch_imgs = []  # 이미지 배열 저장
    x_batch_parts = []  # 부품명 저장
    x_batch_bboxes = []  # bbox 저장

    for i, idx in enumerate(batch):
        img_path = data.iloc[idx, 0]  # 이미지 경로
        label = data.iloc[idx, 1]    # 라벨
        part = data.iloc[idx, 2]     # 부품명 로드
        bbox_str = data.iloc[idx, 3]  # bbox 문자열 로드

        # bbox 문자열을 리스트로 변환
        try:
            bbox = ast.literal_eval(bbox_str)  # 문자열을 리스트로 변환
            if not isinstance(bbox, (list, tuple)):
                raise ValueError  # 리스트나 튜플이 아닌 경우 예외 처리
        except (ValueError, SyntaxError):
            bbox = []  # 변환 실패 시 빈 리스트로 처리

        img = Image.open(img_path)   # 이미지 로드
        img_array = np.array(img)    # 이미지 배열로 변환

        # 데이터 추가
        x_batch_imgs.append(img_array)
        x_batch_parts.append(part)
        x_batch_bboxes.append(bbox)
        y_batch.append(label)        # 배치에 라벨 추가

        # 진행 상황 출력 (배치 내에서만)
        if i % 10 == 0:
            print(f"배치 진행현황: {int(i / len(batch) * 100)}% 완료")

    count += 1  # 배치 완료 횟수 증가
    print(f"{count}회 완료")
    # 각 배치 완료 후 x와 y에 추가
    x.append([np.array(x_batch_imgs), x_batch_parts, x_batch_bboxes])
    y.append(np.array(y_batch))
    

# x와 y를 하나로 결합 (배치 단위로)
x = np.concatenate(x, axis=0)  # 배치 단위로 이미지를 결합
y = np.concatenate(y, axis=0)  # 배치 단위로 라벨을 결합

# 결과 출력
print(f"x.shape: {x.shape}")  # 이미지 배열 크기
print(f"y.shape: {y.shape}")  # 라벨 배열 크기

In [ ]:
# from PIL import Image

# # 이미지 데이터 로드 및 배열 생성
# first_image_path = data.iloc[0, 0]  # 첫 번째 이미지 경로
# img_1 = Image.open(first_image_path)
# a = np.array(img_1)
# x = np.expand_dims(a, axis=0)  # 첫 번째 이미지 배열 확장
# y = []  # 라벨을 저장할 리스트

# # 이미지 데이터 배치 생성
# for i, idx in enumerate(num):
#     img_path = data.iloc[idx, 0]  # 이미지 경로
#     label = data.iloc[idx, 1]    # 라벨
#     img = Image.open(img_path)   # 이미지 로드
#     img_array = np.array(img)    # 이미지 배열로 변환
#     img_expanded = np.expand_dims(img_array, axis=0)  # 차원 확장
#     x = np.concatenate((x, img_expanded), axis=0)     # 이미지 데이터 결합
#     y.append(label)                                   # 라벨 추가

#     # 진행 상황 출력
#     if i % 10 == 0:
#         print(f"진행현황: {int(i / data_num * 100)}% 완료")

# # 첫 번째 이미지 제거
# x = np.delete(x, 0, axis=0)
# y = np.array(y).reshape(data_num, 1)  # 라벨 배열로 변환

# # 결과 출력
# print(f"x.shape: {x.shape}")  # 이미지 배열 크기
# print(f"y.shape: {y.shape}")  # 라벨 배열 크기

In [ ]:
# 열 이름에 따라 데이터 분리
image_paths = data['image_path']  # 이미지 경로
labels = data['label']           # 라벨
part_names = data['part_name']   # 부품 이름
bbox_data = data['bbox']         # bbox 데이터 (문자열 형태로 저장됨)

In [ ]:
import numpy as np
import pickle
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
num = 0
# 저장할 파일 경로
data_save_path = f'./data/sampled_data{num}.pkl'

# 데이터 저장 함수


def save_data(file_path, x, y):
    with open(file_path, 'wb') as f:
        pickle.dump({'x': x, 'y': y}, f)
    print(f"데이터가 '{file_path}'에 저장되었습니다.")

# 데이터 로드 함수


def load_data(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    print(f"데이터가 '{file_path}'에서 로드되었습니다.")
    return data['x'], data['y']

# 데이터 로드 후 샘플링과 분할 처리 (최초 1회 실행)


def process_and_save_data():
    # 이미지와 라벨 데이터 생성 (예시 코드 연결)
    # x와 y는 이미지를 처리한 결과
    x_sampled, y_sampled = resample(x, y, n_samples=100, random_state=42)
    num += 1
    # 데이터 저장
    save_data(data_save_path, x_sampled, y_sampled)

    return x_sampled, y_sampled

# 데이터 로드 및 분할


def load_and_split_data(file_path):
    x_sampled, y_sampled = load_data(file_path)

    # 데이터 분할
    x_training, x_test, y_training, y_test = train_test_split(
        x_sampled, y_sampled, test_size=0.2, shuffle=True, stratify=y_sampled, random_state=34
    )
    x_train, x_valid, y_train, y_valid = train_test_split(
        x_training, y_training, test_size=0.2, shuffle=True, stratify=y_training, random_state=34
    )

    return x_train, x_valid, x_test, y_train, y_valid, y_test

# 데이터 처리 및 저장 (최초 1회 실행)
# x_sampled, y_sampled = process_and_save_data()


# 저장된 데이터 로드 및 분할
x_train, x_valid, x_test, y_train, y_valid, y_test = load_and_split_data(
    data_save_path)

# 결과 출력
print(f"x_train.shape: {x_train.shape}")
print(f"x_valid.shape: {x_valid.shape}")
print(f"x_test.shape: {x_test.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"y_valid.shape: {y_valid.shape}")
print(f"y_test.shape: {y_test.shape}")

# 데이터 연결

In [ ]:
from sklearn.utils import resample
import numpy as np
from sklearn.model_selection import train_test_split


# data sampling
x_sampled, y_sampled = resample(x, y, n_samples=100, random_state=42)

print(x_sampled.shape)
print(y_sampled.shape)


# data split
x_training, x_test, y_training, y_test = train_test_split(
    x_sampled, y_sampled, test_size=0.2, shuffle=True, stratify=y_sampled, random_state=34)
x_train, x_valid, y_train, y_valid = train_test_split(
    x_training, y_training, test_size=0.2, shuffle=True, stratify=y_training, random_state=34)

## 모델 코드 작성

In [ ]:
from ultralytics import YOLO
import cv2
model = YOLO("yolov8s.pt")

## 손실 그래프

In [ ]:
# loss function
import matplotlib.pyplot as plt

# 손실 감소 그래프
plt.subplot(1, 2, 1)
plt.plot(losses)
plt.grid()
plt.title('Loss over Epochs')
plt.xlabel('epoch')
plt.ylabel('Loss')

## 데이터 검증

In [ ]:
## data evaluation

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score
pred=model(y_test)


# 혼동 행렬
confmat = confusion_matrix(y_true=y_test, y_pred=pred)
print('혼동 행렬: ', confmat)


# 정확도
accuracy=accuracy_score(y_true=y_test, y_pred=pred)
print('정확도: ', accuracy)


# 정밀도(precision)
precision = precision_score(y_true=y_test, y_pred=pred)
print('정밀도: ', precision)


# 재현율(recall)
recall = recall_score(y_true=y_test, y_pred=pred)
print('재현율: ', recall)


# roc auc
fpr, tpr, _ = roc_curve(y_true=y_test, y_pred=pred)

plt.plot(fpr, tpr)
plt.xlabel('FP Rate')
plt.ylabel('TP Rate')

plt.show()

print('auc 점수: ', roc_auc_score(y_test, pred))